In [3]:
import os
import json
from glob import glob
os.chdir("..")

In [4]:
path = "/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/calculate_area individual files"
json_list = glob('/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/*.json')
json_list

['/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/zodiac_sign.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/get_weather_info.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/power.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/get_capital.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/calculate_trip_cost.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/count_letter.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/calculate_bmi.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/get_timezone.json',
 '/Users/cs/Desktop/W210/function-calling-project/ks_instruct/dump_json/calculate_area.json']

In [5]:
queries = []
answers = []
for file_path in json_list[0:]:
    with open(file_path, 'r') as file:
        text = json.load(file)
        text = text["queries"]
        for item in text:
            queries.append(item["query"])
            answers.append(json.dumps(item["answer"]))
        

In [6]:
import pandas as pd

# check schema 
df = pd.DataFrame({"queries": queries, "answers": answers})
df["functions"] = df["answers"].apply(lambda x: json.loads(x).get("function", None))

print(f"there are {len(df)} rows at the beginning")

there are 5867 rows at the beginning


In [7]:
#filtered non function
df = df[~df["functions"].isna()]
print(f"there are {len(df)} after filtering out non function")


there are 5866 after filtering out non function


In [8]:
# filter arguments
df["arguments"] = df["answers"].apply(lambda x: json.loads(x).get("arguments", None))
df = df[~df["arguments"].isna()]
# df["arguments"] = df["arguments"].apply(json.dumps)

print(f"there are {len(df)} has arguments keys")


there are 5866 has arguments keys


In [10]:
from src.chi_functions import *
# from src.edward_functions import *
from src.MT_functions import * 
from src.kevin_function import * 


def call_function(info: pd.DataFrame):
    try:
        query = info["queries"]
        func_to_call = eval(info["functions"])
        arguments = json.loads(info["arguments"])
        return {"query": query, "output": func_to_call(**arguments)}
    except:
        return None
df["is_callable"] = df.apply(lambda x: call_function(x), axis=1)



In [11]:
zodiac_sign

,queries,answers,functions,arguments,is_callable
0,What is my zodiac sign if I was born on 1999-0...,"{""function"": ""zodiac_sign"", ""arguments"": {""bir...",zodiac_sign,{'birthday': '1999-02-27'},None
1,What would be the zodiac sign of someone born ...,"{""function"": ""zodiac_sign"", ""arguments"": {""bir...",zodiac_sign,{'birthday': '1985-08-12'},None
2,Is my zodiac sign Aries if I was born on 2001-...,"{""function"": ""zodiac_sign"", ""arguments"": {""bir...",zodiac_sign,{'birthday': '2001-03-22'},None
3,What is the zodiac sign of someone who was bor...,"{""function"": ""zodiac_sign"", ""arguments"": {""bir...",zodiac_sign,{'birthday': '1993-11-16'},None
4,Can you tell me my zodiac sign if I was born o...,"{""function"": ""zodiac_sign"", ""arguments"": {""bir...",zodiac_sign,{'birthday': '1988-06-25'},None
...,...,...,...,...,...
5862,What is the area of a rectangle with length 5....,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{'length': '5.67', 'width': '7.49', 'round_res...",None
5863,What is the area of a rectangle with length 3....,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{'length': '3.19', 'width': '6.75', 'round_res...",None
5864,What is the area of a rectangle with length 1....,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{'length': '1.01', 'width': '8.93', 'round_res...",None
5865,What is the area of a rectangle with length 9....,"{""function"": ""calculate_area"", ""arguments"": {""...",calculate_area,"{'length': '9.82', 'width': '7.49', 'round_res...",None


In [48]:
os.getcwd()

'/Users/cs/Desktop/W210/function-calling-project/notebook'